# Direction of Arrival (DOA) Algorithm Testing

Includes:

- A 2D room simulation to model recording signals in a real environment
- A variety of implementations of (2D) DOA algorithms

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pyroomacoustics as pra
from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)
from pyroomacoustics.doa import circ_dist
from scipy.io import wavfile
from playsound import playsound

### Build Theatre Environment

Create shoebox room of similar size and characteristics to theatre to simulate in.

In [66]:
# build theatre env here

fs = 8000 # audio sampling frequency - global (set to freq of input file)

room_x = 10 # stage width
room_y = 20 # room length
order = 0 # reflection order
sigma2_n = 5e-7 # microphone noise variance estimate
v_sound = 343.0 # speed of sound in air

m = pra.make_materials(
    ceiling="hard_surface",
    floor="stage_floor",
    east="brickwork",
    west="brickwork",
    north="brickwork",
    south="brickwork",
) # materials of each surface of room, defines signal absorption

room = pra.ShoeBox([room_x, room_y], fs=fs, materials=m, max_order=order, air_absorption=True, ray_tracing=False)

### Place microphones

Defines array geometry and places the microphones near the stage (the front of the room).

In [67]:
# design parameters of array
array_loc = [5, 15] # x/y location of array center
n_mics = 16 # number of microphones
sep = 0.1 # microphone separation

N = 1024 # fft length - defines N/2 + 1 frequency bands

R = pra.linear_2D_array(center=array_loc, M=n_mics, phi=0, d=sep)
mic_array = pra.Beamformer(R, room.fs, N) # uses omnidirectional mics by default
room.add_microphone_array(mic_array)

### Place sound sources

Sources can be placed anywhere, with two samples placed apart from each other on stage. The 'input_samples' folder contains all input samples, some taken from the pyroomacoustics github.

In [68]:
# Add sources on stage
source_locs = [[5, 19]] # locations of each source in 3D space (can modify)
source_inputs = ['input_samples/singing_8000.wav', 'input_samples/german_speech_8000.wav'] # filepaths to wav files being played by the source

for i in range(len(source_locs)):
    _, audio = wavfile.read(source_inputs[i])
    room.add_source(source_locs[i], signal=audio, delay=0)

 # ground truth azimuth (calculated relative to the x axis)
azimuth = math.atan2(source_locs[0][1] - array_loc[1], source_locs[0][0] - array_loc[0])
print('Source azimuth:', azimuth*180 / np.pi)

Source azimuth: 90.0


### DOA Algorithms:

Code below tests ability of each algorithm to locate a single source (source: pyroomacoustics/examples/doa_algorithms.py)


In [69]:
room.compute_rir()
room.simulate()

# short-time fourier transform of signal
X = np.array(
    [
        pra.transform.stft.analysis(signal, N, N // 2).T
        for signal in room.mic_array.signals
    ]
)

# algo_names = sorted(pra.doa.algorithms.keys())
# print(algo_names)
# algo_names = ['CSSM', 'FRIDA', 'MUSIC', 'NormMUSIC', 'SRP', 'TOPS', 'WAVES'] # all algorithms
algo_names = ['CSSM', 'MUSIC', 'NormMUSIC', 'SRP', 'TOPS', 'WAVES']

for algo_name in algo_names:
    # Construct the new DOA object
    # the max_four parameter is necessary for FRIDA only
    doa = pra.doa.algorithms[algo_name](R, fs, N, c=v_sound, max_four=4)

    # this call here perform localization on the frames in X
    doa.locate_sources(X, freq_bins=np.arange(1, N))

    # doa.polar_plt_dirac()
    # plt.title(algo_name)

    # doa.azimuth_recon contains the reconstructed location of the source
    print(algo_name)
    print("  Recovered azimuth:", doa.azimuth_recon / np.pi * 180.0, "degrees")
    print("  Angular Error:", circ_dist(azimuth, doa.azimuth_recon) / np.pi * 180.0, "degrees")

plt.show()

CSSM
  Recovered azimuth: [90.] degrees
  Angular Error: [0.] degrees
MUSIC
  Recovered azimuth: [90.] degrees
  Angular Error: [0.] degrees
NormMUSIC
  Recovered azimuth: [90.] degrees
  Angular Error: [0.] degrees
SRP
  Recovered azimuth: [90.] degrees
  Angular Error: [0.] degrees
TOPS
  Recovered azimuth: [270.] degrees
  Angular Error: [180.] degrees
WAVES
  Recovered azimuth: [90.] degrees
  Angular Error: [0.] degrees
